# AWS Public Blockchain - BTC Examples

In [ ]:
pip install PyAthena

In [ ]:
import boto3
import pandas as pd
from pyathena import connect

In [ ]:
S3_BUCKET=!(aws cloudformation list-exports --query "Exports[?Name=='public-blockchain-data-athena'].Value" --output text)
S3_BUCKET=S3_BUCKET[0]

In [ ]:
conn = connect(s3_staging_dir='s3://'+S3_BUCKET+'/results/')

### Get number of blocks

In [ ]:
df = pd.read_sql("select count(*) as block_count from btc.blocks", conn)
df

### Get number of transactions per date

In [ ]:
df = pd.read_sql("select date,sum(transaction_count) as tx_count from btc.blocks group by date order by date", conn)
df

### Get details for block by number

In [ ]:
block_number=700000

In [ ]:
df = pd.read_sql("select * from btc.blocks where number="+str(block_number), conn)
df.iloc[0]

### Get details for transaction by hash

In [ ]:
tx_hash='c86aecd51212a81bfdb26077aaceb1d9822802bb48fb542e41502487060d7828'

In [ ]:
df = pd.read_sql("select * from btc.transactions where hash='"+tx_hash+"'", conn)
df.iloc[0]

### Get inputs by transaction hash

In [ ]:
df = pd.read_sql("SELECT t.hash,"+
                 "input.index as input_index,input.script_asm as input_script_asm,input.script_hex as input_script_hex,input.sequence as input_sequence,input.required_signatures as inputrequired_signatures,input.type as input_type,input.address as input_address,input.value as input_value from btc.transactions t,UNNEST(t.inputs) as t(input) "+
                 "where t.hash='"+tx_hash+"' order by t.hash,input.index", conn)
df

### Get outputs by transaction hash

In [ ]:
df = pd.read_sql("SELECT t.hash,"+
                 "output.index as output_index,output.script_asm as output_script_asm,output.script_hex as output_script_hex,output.required_signatures as output_required_signatures,output.type as output_type,output.address as output_address,output.value as output_value from btc.transactions t,UNNEST(t.outputs) as t(output) "+
                 "where t.hash='"+tx_hash+"' order by t.hash,output.index", conn)
df

### Get transaction inputs and outputs by transaction hash

In [ ]:
df = pd.read_sql("SELECT A.*,B.input_index,B.input_script_asm,B.input_script_hex,B.input_type,B.input_address,B.input_value FROM (SELECT t.hash,t.size,t.virtual_size,t.version,t.lock_time,t.block_hash,t.block_number,t.block_timestamp,t.input_count,t.output_count,t.input_value,t.output_value,t.is_coinbase,t.fee,"+
                 "output.index as output_index,output.script_asm as output_script_asm,output.script_hex as output_script_hex,output.required_signatures as output_required_signatures,output.type as output_type,output.address as output_address,output.value as output_value "+
                 "from btc.transactions t,UNNEST(t.outputs) as t(output) "+
                 "where t.hash='"+tx_hash+"') A FULL OUTER JOIN "+
                 "(SELECT t.hash,"+
                 "input.index as input_index,input.script_asm as input_script_asm,input.script_hex as input_script_hex,input.type as input_type,input.address as input_address,input.value as input_value from btc.transactions t,UNNEST(t.inputs) as t(input) "+
                 "where t.hash='"+tx_hash+"' order by t.hash,input.index) B ON A.hash=B.hash and A.output_index=B.input_index order by A.hash,A.output_index", conn)
df

## Charts

### Fees over Time (in BTC)

In [ ]:
%matplotlib inline
df = pd.read_sql("select date,sum(fee) as fees from btc.transactions where date>='2018-06-01' and date<='2021-01-01' group by date", conn)
df.set_index(pd.DatetimeIndex(df['date']),inplace=True)
del df['date']
df["fees"].plot(figsize=(15,8))

### Transactions per Second

In [ ]:
%matplotlib inline
df = pd.read_sql("""
select date_trunc('month',cast(date as date)) as date, avg(transactions_per_sec) as transactions_per_sec from (
    select date, cast(count(*) as double)/(24*60*60) as transactions_per_sec
    from btc.transactions
    where date>'2015-01-01'
    group by date) group by  date_trunc('month',cast(date as date))
""", conn)
df.set_index(pd.DatetimeIndex(df['date']),inplace=True)
del df['date']
df["transactions_per_sec"].plot(figsize=(15,8))

### Historical Price Data 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_sql("""
SELECT CAST(open_time AS DATE) as date,close FROM marketdata.crypto where ticker='BTCUSDT' and extract(MINUTE from open_time)=59 and extract(HOUR from open_time)=23 and open_time > TIMESTAMP '2021-01-01 00:00:00' order by open_time
""", conn)
df.set_index(pd.DatetimeIndex(df['date']),inplace=True)
del df['date']

plt.rcParams["figure.figsize"] = (15,8)
plt.plot(df.index, df['close'], color='orange')
plt.title('Historical BTC Price', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.ylabel('USD', fontsize=14)
plt.grid(True)
plt.show()

### Total Supply over Time

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#(CASE WHEN number<210000 THEN 50 WHEN number<420000 THEN 25 WHEN number<630000 THEN 12.5 ELSE 6.25 END)
df = pd.read_sql(""" 
select date, (CASE 
  WHEN number<210000 THEN 50*number
  WHEN number<420000 THEN (210000-1)*50+25*(number-210000) 
  WHEN number<630000 THEN (210000-1)*50+25*210000+12.5*(number-420000)
  ELSE (210000-1)*50+25*210000+12.5*210000+6.25*(number-630000) 
  END) as reward
from (select cast(date as date) as date,max(number) as number from btc.blocks group by date) order by date 
""", conn)
df.set_index(pd.DatetimeIndex(df['date']),inplace=True)
del df['date']

plt.rcParams["figure.figsize"] = (15,8)
plt.plot(df.index, df['reward'], color='blue')
plt.title('Total Supply Over Time', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.ylabel('BTC', fontsize=14)
plt.grid(True)
plt.show()

### Weekly Transaction Volume (USD) 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_sql("""
WITH 
  daily_amt AS (SELECT cast(date AS date) AS date, sum(input_value) AS amt FROM btc. transactions where date>='2018-01-01' GROUP BY date),
  daily_price as (SELECT cast(open_time AS DATE) AS date, close AS price FROM marketdata.crypto where ticker='BTCUSDT' and extract(MINUTE FROM open_time)=59 and extract(HOUR FROM open_time)=23 and open_time >= TIMESTAMP '2020-01-01 00:00:00')
SELECT date_trunc('week', date) AS week, sum(transaction_volume) / 1000000000 AS transaction_volume, avg(price) AS price FROM (SELECT A.date, B.amt * A.price AS transaction_volume, a.price FROM daily_price A JOIN daily_amt B ON A.date=B.date) GROUP BY date_trunc('week', date) ORDER BY date_trunc('week',date) 
""", conn)
df.set_index(pd.DatetimeIndex(df['week']),inplace=True)

plt.rcParams["figure.figsize"] = (13,6)
plt.plot(df.index, df['transaction_volume'], color='blue')
plt.title('Weekly Transaction Volume', fontsize=14)
plt.xlabel('Week', fontsize=14)
plt.ylabel('Volume', fontsize=14)
plt.grid(True)
plt.show()

### Network Value to Transactions (NVT) Ratio

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_sql("""
with daily_amt as 
  (select cast(date as date) as date,sum(input_value) as amt from btc.transactions where date>='2020-01-01' group by date),
     daily_price as
  (SELECT CAST(open_time AS DATE) as date,close as price FROM marketdata.crypto where ticker='BTCUSDT' and extract(MINUTE from open_time)=59 and extract(HOUR from open_time)=23 and open_time >= TIMESTAMP '2020-01-01 00:00:00'),
    supply_amt as 
  (select date, (CASE 
  WHEN number<210000 THEN 50*number
  WHEN number<420000 THEN (210000-1)*50+25*(number-210000) 
  WHEN number<630000 THEN (210000-1)*50+25*210000+12.5*(number-420000)
  ELSE (210000-1)*50+25*210000+12.5*210000+6.25*(number-630000) 
  END) as supply
  from (select cast(date as date) as date,max(number) as number from btc.blocks group by date) order by date) 
select A.date,(C.supply*A.price)/(B.amt*A.price) as nvt from daily_price A JOIN daily_amt B ON A.date=B.date JOIN supply_amt C ON B.date=C.date order by A.date
""", conn)
df.set_index(pd.DatetimeIndex(df['date']),inplace=True)
del df['date']

plt.rcParams["figure.figsize"] = (15,8)
plt.plot(df.index, df['nvt'], color='blue')
plt.title('Network Value to Transactions (NVT) Ratio', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Ratio', fontsize=14)
plt.grid(True)
plt.show()